# Feasibility Pump example

The Feasibility Pump is the heuristic is responsible for the idea to this module and therefore the core heuristic of HIPS. First proposed in 2005 by Fischetti et al. [[FGL05]](https://cxlvinchau.github.io/hips/references.html#id4), the Feasibility Pump aims to compute a feasible solution to an MIP without the need for an initial solution. The heuristic is also described in [[Ber06]](https://cxlvinchau.github.io/hips/references.html#id2).

## Installing the packages

In the following we use the `ClpSolver` and thus need to install various binaries and CyLP.

In [ ]:
%%capture
!sudo apt-get install -y git pkg-config coinor-libcbc-dev coinor-libosi-dev coinor-libcoinutils-dev coinor-libcgl-dev
!export COIN_INSTALL_DIR=/usr/
!pip install --pre cylp

Afterwards, we can install HIPS.

In [ ]:
%%capture
!pip install https://github.com/cxlvinchau/hips/archive/master.zip

## Example

First add the required imports:

In [ ]:
from hips import load_problem, ProblemSense
from hips.heuristics import FeasibilityPump

Now let us load a problem. In this example we load the `10teams` example from MIPLIB 2017. The example can be found [here](https://miplib.zib.de/instance_details_10teams.html).

In [ ]:
mip_model = load_problem("10teams")
mip_model.summary()

Now let us create Feasibility Pump and specify the iteration maximum `max_iter` to 1000 and the perturbation factor `t` to 15:

In [ ]:
heur = FeasibilityPump(mip_model, t=15)
heur.compute(max_iter=1000)

Now let us inspect the solution found:

In [ ]:
import matplotlib
%matplotlib inline
print("Status: {}".format(heur.get_status()))
print("Found solution: {}".format(heur.get_objective_value()))
heur.tracker.plot("feasibility objective")

### Playing with the parameter 

The Feasibility Pump can achieve quite different solutions, depending on the parameter `t`. 
This parameter represent how many variables we perturb, in case a cycle is detected. Let us therefore play with its values.
To avoid different outcomes in the following experiments, we will furthermore set a seed for the Feasibility Pump.

In [ ]:
import time
import numpy as np

We will start of with a high value for `t`:

In [ ]:
print("#-------- Start experiment 1 --------#")

feasible = []
times = []
test_number = 5
for i in range(test_number):
    mip_model = load_problem("10teams")

    mip_model.set_mip_sense(ProblemSense.MIN)
    start = time.time()

    fs = FeasibilityPump(mip_model, t=30, seed=i)
    fs.compute(1000)

    end = time.time()

    sol = {var: fs.variable_solution(var) for var in mip_model.lp_model.vars}
    feasible.append(mip_model.is_feasible(sol))

    times.append(end-start)
    print("Model {} solved.".format(i))

print("#----------------------------------#")
print("Feasible solutions found in {} tests: {}".format(test_number, sum(feasible)))

print(times)
print("Average time consumed: {}s".format(sum(times)/len(times)))

times = np.array(times)
feasible_times = times[np.array(feasible)]
print("Average time consumed for feasible solutions: {}s".format(sum(feasible_times)/len(feasible_times)))


We should observe, that every run of the Feasibility Pump has found a solution. The average runtime is rather high.

Now we will do the same computations with a lower value for `t`:

In [ ]:
print("#-------- Start experiment 2 --------#")

feasible = []
times = []
test_number = 5
for i in range(test_number):
    mip_model = load_problem("10teams")

    mip_model.set_mip_sense(ProblemSense.MIN)
    start = time.time()

    fs = FeasibilityPump(mip_model, t=6, seed=i)
    fs.compute(1000)

    end = time.time()

    sol = {var: fs.variable_solution(var) for var in mip_model.lp_model.vars}
    feasible.append(mip_model.is_feasible(sol))

    times.append(end-start)
    print("Model {} solved.".format(i))

print("#----------------------------------#")
print("Feasible solutions found in {} tests: {}".format(test_number, sum(feasible)))

print(times)
print("Average time consumed: {}s".format(sum(times)/len(times)))

times = np.array(times)
feasible_times = times[np.array(feasible)]
print("Average time consumed for feasible solutions: {}s".format(sum(feasible_times)/len(feasible_times)))

We should now observe, that not every computation has found a solution. Nevertheless, the computations that found a solution are quite a bit faster than the runs from Experiment 1.

We encourage you now to play with the parameter of the Feasibility Pump to get more acquainted with the significance of the choice of the optimal value for the parameter `t`.